In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bagssss/watchesbagsaccessories.csv


In [2]:
import pandas as pd
import numpy as np

missing_value = ["N/A", "na", np.nan]
df = pd.read_csv("/kaggle/input/bagssss/watchesbagsaccessories.csv", na_values=missing_value, encoding='latin1')


In [3]:
df.head()

,Title,Rating in Stars,Rating Count,Sold Count,Voucher,Delivery,Currency,Current Price,Original Price,Link,Category
0,Yfashion Cartoon Electronic Watch Life Water...,4.6/5,707,6K Sold,0,Free Delivery,Rs.,287,Rs. 520,//www.daraz.pk/products/y-led-3-i258744118-s14...,Kids Watches
1,"100% Imported LED watch for men , boys and Kid...",4.3/5,293,1K Sold,0,0,Rs.,270,Rs. 999,//www.daraz.pk/products/100-led-2023-i40167553...,Kids Watches
2,1 Cartoon Characters Analog Wrist Watch For Ki...,4.5/5,57,452 Sold,0,Free Delivery,Rs.,225,Rs. 750,//www.daraz.pk/products/1-i423737473-s20070599...,Kids Watches
3,M3 Touch LED Bracelet Digital Watch Band Good ...,4.6/5,51,332 Sold,0,Free Delivery,Rs.,160,Rs. 299,//www.daraz.pk/products/m3-led-led-i432456407-...,Kids Watches
4,led Watches For Boys and Girls and kids,4.4/5,54,476 Sold,0,Free Delivery,Rs.,198,Rs. 300,//www.daraz.pk/products/led-i398157365-s192654...,Kids Watches


In [4]:
df.isnull().sum()

Title               0
Rating in Stars     0
Rating Count        0
Sold Count         34
Voucher             0
Delivery            0
Currency            0
Current Price       0
Original Price     68
Link                0
Category            0
dtype: int64

In [5]:
df['Sold Count'].fillna(0, inplace=True)

df['Original Price'].fillna(df['Current Price'], inplace=True)

/tmp/ipykernel_33/215041037.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Sold Count'].fillna(0, inplace=True)
/tmp/ipykernel_33/215041037.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.meth

In [6]:
df.isnull().sum()

Title              0
Rating in Stars    0
Rating Count       0
Sold Count         0
Voucher            0
Delivery           0
Currency           0
Current Price      0
Original Price     0
Link               0
Category           0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            1078 non-null   object
 1   Rating in Stars  1078 non-null   object
 2   Rating Count     1078 non-null   int64 
 3   Sold Count       1078 non-null   object
 4   Voucher          1078 non-null   object
 5   Delivery         1078 non-null   object
 6   Currency         1078 non-null   object
 7   Current Price    1078 non-null   object
 8   Original Price   1078 non-null   object
 9   Link             1078 non-null   object
 10  Category         1078 non-null   object
dtypes: int64(1), object(10)
memory usage: 92.8+ KB


In [8]:
df = df[~df['Rating in Stars'].str.contains(r'\d{1,2}-[A-Za-z]{3}', na=False)]
df['Rating in Stars'] = df['Rating in Stars'].str.replace('/5', '')
df['Rating in Stars'] = df['Rating in Stars'].astype(float)

In [9]:
# df['Sold Count'] = df['Sold Count'].astype(str)
# df['Sold Count'] = df['Sold Count'].str.replace(r'\D', '', regex=True)
# df.loc[df['Sold Count'].str.contains('k', na=False), 'Sold Count'] = df.loc[df['Sold Count'].str.contains('k', na=False), 'Sold Count'].str.replace('k', '').astype(float) * 1000

In [10]:
df['Sold Count'] = df['Sold Count'].astype(str)
df['Sold Count'] = df['Sold Count'].str.replace(r'\D', '', regex=True)
df['Sold Count'] = df['Sold Count'].str.replace(',', '')
df.loc[df['Sold Count'].str.contains('k', na=False), 'Sold Count'] = df.loc[df['Sold Count'].str.contains('k', na=False), 'Sold Count'].str.replace('k', '').astype(float) * 1000
df['Sold Count'] = df['Sold Count'].astype(float)
df['Sold Count'] = df['Sold Count'].apply(lambda x: x if isinstance(x, float) else None)


In [11]:
categorical_cols = ['Category', 'Voucher', 'Delivery', 'Currency']
df_encoded = pd.get_dummies(df, columns=categorical_cols)

In [12]:
X = df_encoded.drop(columns=['Title', 'Link'])
y = df_encoded['Sold Count']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

ValueError: could not convert string to float: '1,059'

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
feature_importance.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
future_predictions = model.predict(X_future)

In [ ]:
plt.plot(X_future['Date'], future_predictions, label='Future Predictions')
plt.xlabel('Date')
plt.ylabel('Sold Count')
plt.title('Future Sales Forecast')
plt.legend()
plt.show()